<a href="https://colab.research.google.com/github/PaavaniKohli/PRODIGY_ML_03/blob/main/Cats_Dogs_Paavani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Implement a support vector machine (svm) to classify images of cats and dogs from a kaggle data set

In [12]:
!pip install numpy scikit-learn opencv-python
import os
from skimage.feature import hog
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report

# Define the path to the dataset
dataset_path = '/content/drive/MyDrive/train'

# Read images and labels
images = []
labels = []

for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)

      # Check if the category_path is a directory
    if not os.path.isdir(category_path):
        continue

    for file_name in os.listdir(category_path):
        image_path = os.path.join(category_path, file_name)

        # Check if the image_path is a file
        if not os.path.isfile(image_path):
            continue

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        image = cv2.resize(image, (64, 64))  # Resize for consistency

        # Extract HOG features
        features, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)

        images.append(features)  # Use HOG features instead of flattened image
        labels.append(category)

# Convert lists to NumPy arrays
X = np.array(images)
y = np.array(labels)

# Normalize pixel values to [0, 1]
X = X / 255.0

# Flatten images for SVM input
X_flatten = X.reshape(X.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_flatten, y, test_size=0.2, random_state=42)

# Define the parameter grid for grid search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
}

# Create an SVM classifier
svm_model = SVC(random_state=42)

# Perform grid search
grid_search = GridSearchCV(svm_model, param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters found by grid search
print("Best Parameters:", grid_search.best_params_)

# Make predictions on the test set using the best model
y_pred = grid_search.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print(classification_report(y_test, y_pred, target_names=['cat', 'dog']))


Best Parameters: {'C': 10, 'kernel': 'rbf'}
Accuracy: 0.8316970546984572
              precision    recall  f1-score   support

         cat       0.86      0.94      0.90      1116
         dog       0.68      0.43      0.52       310

    accuracy                           0.83      1426
   macro avg       0.77      0.69      0.71      1426
weighted avg       0.82      0.83      0.82      1426

